# Multi-Agent Loan Application Assistant

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/robin-mongodb/mongochain/blob/main/examples/verticals/banking/loan_application_assistant.ipynb)

A multi-agent system demonstrating agent-to-agent collaboration for loan processing.

## Architecture

```
┌─────────────────────────────────────────────────────────────────────┐
│                           User Interaction                          │
└─────────────────────────────┬───────────────────────────────────────┘
                              │
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                    Agent A: Loan Interviewer                        │
│  • User-facing, friendly persona                                    │
│  • Gathers: income, employment, loan amount, purpose                │
│  • Validates application completeness                               │
│  • Hands off to Underwriter via tool                                │
└─────────────────────────────┬───────────────────────────────────────┘
                              │  Structured JSON handoff
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│                    Agent B: Loan Underwriter                        │
│  • Technical, analytical persona                                    │
│  • Checks credit score, calculates debt-to-income ratio             │
│  • Returns: Approved / Denied / Manual Review + rationale           │
└─────────────────────────────┬───────────────────────────────────────┘
                              │  Decision + rationale
                              ▼
┌─────────────────────────────────────────────────────────────────────┐
│             Interviewer translates to friendly response             │
└─────────────────────────────────────────────────────────────────────┘
```

## Features Demonstrated
- **Multi-Agent Collaboration**: Two specialized agents working together
- **Memory Sharing via `collaborators`**: Agents can read each other's memories
- **Tool-Based Handoff**: Structured task delegation between agents
- **Persona Specialization**: Friendly interviewer vs. analytical underwriter

## Prerequisites
- MongoDB Atlas cluster
- Voyage AI API key
- OpenAI/Gemini/Claude API key

## 1. Setup

In [ ]:
# Install mongochain from GitHub
%pip install -q git+https://github.com/robin-mongodb/mongochain.git

In [ ]:
from getpass import getpass

# API Keys (input hidden)
MONGO_URI = getpass("MongoDB URI: ")
VOYAGE_API_KEY = getpass("Voyage API Key: ")
OPENAI_API_KEY = getpass("OpenAI API Key: ")

## 2. Seed Mock User Credit Data

Create mock credit profiles for demo users. In a real system, this would come from a credit bureau API.

In [ ]:
from pymongo import MongoClient
from datetime import datetime

# Connect to MongoDB
client = MongoClient(MONGO_URI)
db = client["loan_underwriter"]  # Underwriter's database
credit_profiles = db["credit_profiles"]

# Clear existing demo data
credit_profiles.delete_many({"user_id": {"$in": ["sarah.johnson@email.com", "mike.chen@email.com"]}})

# Mock credit profiles
MOCK_PROFILES = [
    {
        "user_id": "sarah.johnson@email.com",
        "credit_score": 742,
        "annual_income": 95000,
        "monthly_debt_payments": 1200,  # Car loan, student loans
        "employment_status": "full_time",
        "employment_years": 6,
        "existing_loans": [
            {"type": "auto", "balance": 12000, "monthly_payment": 450},
            {"type": "student", "balance": 25000, "monthly_payment": 750}
        ],
        "created_at": datetime.now()
    },
    {
        "user_id": "mike.chen@email.com",
        "credit_score": 580,
        "annual_income": 45000,
        "monthly_debt_payments": 1800,  # Credit cards, personal loan
        "employment_status": "part_time",
        "employment_years": 1,
        "existing_loans": [
            {"type": "credit_card", "balance": 8000, "monthly_payment": 400},
            {"type": "personal", "balance": 15000, "monthly_payment": 600},
            {"type": "credit_card", "balance": 5000, "monthly_payment": 800}
        ],
        "created_at": datetime.now()
    }
]

credit_profiles.insert_many(MOCK_PROFILES)
print(f"✅ Inserted {len(MOCK_PROFILES)} credit profiles")

# Verify insertion
verify_count = credit_profiles.count_documents({})
print(f"✅ Verified: {verify_count} profiles in database")

# Display profiles
for profile in MOCK_PROFILES:
    dti = (profile["monthly_debt_payments"] / (profile["annual_income"] / 12)) * 100
    print(f"\n📊 {profile['user_id']}")
    print(f"   Credit Score: {profile['credit_score']}")
    print(f"   Annual Income: ${profile['annual_income']:,}")
    print(f"   Monthly Debt: ${profile['monthly_debt_payments']:,}")
    print(f"   Current DTI: {dti:.1f}%")
    print(f"   Employment: {profile['employment_status']} ({profile['employment_years']} years)")

## 3. Create Agent B: The Loan Underwriter

The Underwriter is created first because the Interviewer needs to reference it.

**Role**: Analytical decision-maker that evaluates loan applications based on credit rules.

In [ ]:
from mongochain import MongoAgent

underwriter = MongoAgent(
    name="loan_underwriter",
    persona="""You are a Loan Underwriter AI assistant.

Your role is to evaluate loan applications based on credit data and financial metrics.

IMPORTANT - TOOL USAGE:
- You have ONE tool: 'evaluate_loan_application' that retrieves credit data AND calculates DTI
- ALWAYS use this tool with the applicant's user_id and loan_amount to get all data needed for a decision
- NEVER ask the user for credit scores, income, or debt information - this data is in the database
- You MUST call the evaluate_loan_application tool before making any decision

Decision Criteria (apply AFTER retrieving credit data):
- Credit Score >= 700 AND DTI <= 36%: APPROVED
- Credit Score 620-699 OR DTI 36-43%: MANUAL_REVIEW (needs human review)
- Credit Score < 620 OR DTI > 43%: DENIED

Additional factors to consider:
- Employment stability (years at current job)
- Loan amount relative to income
- Existing debt load

Always provide:
1. A clear decision (APPROVED, DENIED, or MANUAL_REVIEW)
2. Key metrics used in the decision (credit score, DTI ratio)
3. Specific rationale for the decision
4. Any conditions or recommendations

Be precise, analytical, and objective in your assessments.""",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=OPENAI_API_KEY,
    llm_provider="openai",
    collaborators=["loan_interviewer"]  # Can see what Interviewer learned about user
)

underwriter.set_description("Analytical loan underwriting agent that evaluates applications based on credit rules.")
print(f"\n{underwriter}")

### Register Underwriter Tools

Tools for credit checking and DTI calculation.

In [ ]:
import json
from pymongo import MongoClient

# Store the URI in a variable that will be captured by the closure
_MONGO_URI_FOR_TOOLS = MONGO_URI

# Helper function to get fresh database connection each time
def _get_credit_profiles_collection():
    """Get a fresh connection to the credit_profiles collection."""
    client = MongoClient(_MONGO_URI_FOR_TOOLS)
    return client["loan_underwriter"]["credit_profiles"]


def evaluate_loan_application(user_id: str, loan_amount: float, loan_term_months: int = 60, interest_rate: float = 8.5) -> str:
    """
    Comprehensive loan evaluation tool that retrieves credit profile AND calculates DTI.
    Returns all information needed to make a lending decision.
    """
    print(f"    🔍 DEBUG: evaluate_loan_application called for user_id='{user_id}', amount=${loan_amount}")
    
    # Get fresh connection
    credit_profiles = _get_credit_profiles_collection()
    
    # Get the profile
    profile = credit_profiles.find_one({"user_id": user_id})
    
    if not profile:
        print(f"    ❌ DEBUG: No profile found for '{user_id}'")
        return json.dumps({"error": f"No credit profile found for {user_id}"})
    
    print(f"    ✅ DEBUG: Found profile for '{user_id}'")
    
    # Calculate monthly payment for new loan
    monthly_rate = (interest_rate / 100) / 12
    if monthly_rate > 0:
        new_monthly_payment = loan_amount * (monthly_rate * (1 + monthly_rate)**loan_term_months) / ((1 + monthly_rate)**loan_term_months - 1)
    else:
        new_monthly_payment = loan_amount / loan_term_months
    
    # Calculate DTI
    monthly_income = profile["annual_income"] / 12
    current_debt = profile["monthly_debt_payments"]
    total_debt_with_new = current_debt + new_monthly_payment
    
    current_dti = (current_debt / monthly_income) * 100
    new_dti = (total_debt_with_new / monthly_income) * 100
    
    # Build comprehensive result
    result = {
        "applicant": {
            "user_id": user_id,
            "credit_score": profile["credit_score"],
            "annual_income": profile["annual_income"],
            "monthly_income": round(monthly_income, 2),
            "employment_status": profile["employment_status"],
            "employment_years": profile["employment_years"],
            "existing_monthly_debt": current_debt
        },
        "loan_request": {
            "amount": loan_amount,
            "term_months": loan_term_months,
            "interest_rate": interest_rate,
            "estimated_monthly_payment": round(new_monthly_payment, 2)
        },
        "risk_assessment": {
            "current_dti_percent": round(current_dti, 1),
            "new_dti_percent": round(new_dti, 1),
            "dti_increase": round(new_dti - current_dti, 1),
            "credit_score_tier": "excellent" if profile["credit_score"] >= 750 else "good" if profile["credit_score"] >= 700 else "fair" if profile["credit_score"] >= 650 else "poor" if profile["credit_score"] >= 600 else "very_poor"
        },
        "decision_factors": {
            "credit_score_passes": profile["credit_score"] >= 700,
            "dti_under_36": new_dti <= 36,
            "dti_under_43": new_dti <= 43,
            "stable_employment": profile["employment_years"] >= 2
        }
    }
    
    print(f"    📊 DEBUG: Credit score={profile['credit_score']}, New DTI={new_dti:.1f}%")
    
    return json.dumps(result)


# Verify data exists before registering
_test_coll = _get_credit_profiles_collection()
_count = _test_coll.count_documents({})
print(f"📊 Found {_count} credit profiles in database")
if _count == 0:
    print("⚠️  WARNING: No credit profiles found! Make sure to run the 'Seed Mock Data' cell first.")

# Register the comprehensive evaluation tool
underwriter.register_tool(
    name="evaluate_loan_application",
    func=evaluate_loan_application,
    description="Evaluate a loan application by retrieving the applicant's credit profile and calculating DTI with the new loan. Returns all information needed to make a lending decision including credit score, DTI ratios, and decision factors.",
    parameters={
        "type": "object",
        "properties": {
            "user_id": {
                "type": "string",
                "description": "The applicant's email/ID"
            },
            "loan_amount": {
                "type": "number",
                "description": "The requested loan amount in dollars"
            },
            "loan_term_months": {
                "type": "integer",
                "description": "Loan term in months (default: 60)"
            },
            "interest_rate": {
                "type": "number",
                "description": "Annual interest rate as percentage (default: 8.5)"
            }
        },
        "required": ["user_id", "loan_amount"]
    }
)

## 4. Create Agent A: The Loan Interviewer

**Role**: User-facing agent that gathers application details and communicates decisions in a friendly manner.

In [ ]:
interviewer = MongoAgent(
    name="loan_interviewer",
    persona="""You are a friendly Loan Application Assistant.

Your role is to help users apply for personal loans by:
1. Gathering necessary information (loan amount, purpose, employment details)
2. Ensuring the application is complete before submission
3. Submitting to the underwriting team for a decision
4. Explaining decisions in a clear, empathetic way

Information to collect:
- Desired loan amount
- Purpose of the loan
- Employment status (full-time, part-time, self-employed, unemployed)
- How long at current job
- Any additional context the user wants to share

Guidelines:
- Be warm, professional, and reassuring
- Ask one or two questions at a time, not all at once
- Confirm details before submitting
- When delivering decisions, be empathetic especially for denials
- For denials, focus on what the user CAN do to improve their chances
- Never share raw technical details (DTI ratios, exact scores) - translate to plain language""",
    mongo_uri=MONGO_URI,
    voyage_api_key=VOYAGE_API_KEY,
    llm_api_key=OPENAI_API_KEY,
    llm_provider="openai",
    collaborators=["loan_underwriter"]  # Can see Underwriter's past decisions
)

interviewer.set_description("User-facing loan application assistant that gathers information and communicates decisions.")
print(f"\n{interviewer}")

### Register Interviewer Tools

The key tool here is `submit_to_underwriter` which performs the **agent-to-agent handoff**.

In [ ]:
import json

def submit_to_underwriter(
    applicant_id: str,
    loan_amount: float,
    loan_purpose: str,
    employment_status: str,
    years_employed: int
) -> str:
    """Submit a loan application to the Underwriter agent for a decision.
    
    This function performs the agent-to-agent handoff by:
    1. Packaging the application data as structured JSON
    2. Calling the Underwriter agent's chat method
    3. Returning the Underwriter's decision
    """
    # Package the application as structured data
    application = {
        "applicant_id": applicant_id,
        "loan_amount": loan_amount,
        "loan_purpose": loan_purpose,
        "employment_status": employment_status,
        "years_employed": years_employed,
        "submitted_at": datetime.now().isoformat()
    }
    
    # Create the underwriting request - explicitly ask to use tools
    underwriting_request = f"""Evaluate this loan application using your tools.

Application Details:
{json.dumps(application, indent=2)}

REQUIRED STEPS:
1. Use the evaluate_loan_application tool with user_id="{applicant_id}" and loan_amount={loan_amount}
2. Review the credit score and DTI ratios from the tool result
3. Apply decision criteria and return: APPROVED, DENIED, or MANUAL_REVIEW
4. Include the credit score, DTI ratio, and detailed rationale in your response
"""
    
    # AGENT-TO-AGENT HANDOFF: Call the Underwriter agent
    print(f"\n🔄 Handing off to Underwriter agent...")
    print(f"   Application: ${loan_amount:,.2f} for {loan_purpose}")
    
    underwriter_response = underwriter.chat(
        user_id=applicant_id,
        message=underwriting_request
    )
    
    print(f"✅ Underwriter decision received")
    
    return underwriter_response


interviewer.register_tool(
    name="submit_to_underwriter",
    func=submit_to_underwriter,
    description="""Submit a completed loan application to the Underwriter for a credit decision. 
Use this ONLY after collecting all required information from the user. 
Returns the underwriter's decision and rationale.""",
    parameters={
        "type": "object",
        "properties": {
            "applicant_id": {
                "type": "string",
                "description": "The applicant's email/user ID"
            },
            "loan_amount": {
                "type": "number",
                "description": "Requested loan amount in dollars"
            },
            "loan_purpose": {
                "type": "string",
                "description": "Purpose of the loan (e.g., 'home improvement', 'debt consolidation')"
            },
            "employment_status": {
                "type": "string",
                "enum": ["full_time", "part_time", "self_employed", "unemployed"],
                "description": "Current employment status"
            },
            "years_employed": {
                "type": "integer",
                "description": "Years at current job"
            }
        },
        "required": ["applicant_id", "loan_amount", "loan_purpose", "employment_status", "years_employed"]
    }
)

print("✅ Interviewer tools registered!")

## 5. Demo: Successful Loan Application

Sarah has good credit (742) and stable employment. Let's walk through her application.

**User Goal**: Apply for a $10,000 personal loan for home improvement.

In [ ]:
# User: Sarah Johnson (good credit profile)
USER_ID = "sarah.johnson@email.com"

# Start the conversation
response = interviewer.chat(
    user_id=USER_ID,
    message="Hi! I'm Sarah Johnson (sarah.johnson@email.com). I'd like to apply for a personal loan of $10,000 for some home improvements."
)

print(response)

In [ ]:
# Provide employment details
response = interviewer.chat(
    user_id=USER_ID,
    message="I work full-time as a software engineer. I've been at my current company for 6 years."
)

print(response)

In [ ]:
# Confirm and submit
response = interviewer.chat(
    user_id=USER_ID,
    message="Yes, that's all correct. Please go ahead and submit my application!"
)

print(response)

## 6. Demo: Denied Loan Application

Mike has lower credit (580) and high existing debt. Let's see how the system handles a denial.

**User Goal**: Apply for a $15,000 personal loan for debt consolidation.

In [ ]:
# User: Mike Chen (poor credit profile)
USER_ID_2 = "mike.chen@email.com"

# Start the conversation
response = interviewer.chat(
    user_id=USER_ID_2,
    message="Hi, I'm Mike Chen (mike.chen@email.com). I need a $15,000 loan to consolidate my credit card debt. Is that possible?"
)

print(response)

In [ ]:
# Provide employment details
response = interviewer.chat(
    user_id=USER_ID_2,
    message="I'm working part-time right now. Been at this job for about a year."
)

print(response)

In [ ]:
# Submit application
response = interviewer.chat(
    user_id=USER_ID_2,
    message="Yes, please submit the application. I really need this loan."
)

print(response)

## 7. View Memory Sharing Between Agents

Because both agents list each other as `collaborators`, they can access each other's memories about the user.

Let's see what each agent has learned and stored.

In [ ]:
# End sessions to save short-term memories
interviewer.end_session("sarah.johnson@email.com")
interviewer.end_session("mike.chen@email.com")
underwriter.end_session("sarah.johnson@email.com")
underwriter.end_session("mike.chen@email.com")

In [ ]:
print("=" * 60)
print("INTERVIEWER AGENT - What it knows about Sarah")
print("=" * 60)

interviewer_memories = interviewer.get_user_memories("sarah.johnson@email.com", limit=5)
for m in interviewer_memories:
    print(f"  [{m.get('category', 'general')}] {m['content']}")

print("\n" + "=" * 60)
print("UNDERWRITER AGENT - What it knows about Sarah")
print("=" * 60)

underwriter_memories = underwriter.get_user_memories("sarah.johnson@email.com", limit=5)
for m in underwriter_memories:
    print(f"  [{m.get('category', 'general')}] {m['content']}")

In [ ]:
print("=" * 60)
print("INTERVIEWER AGENT - What it knows about Mike")
print("=" * 60)

interviewer_memories = interviewer.get_user_memories("mike.chen@email.com", limit=5)
for m in interviewer_memories:
    print(f"  [{m.get('category', 'general')}] {m['content']}")

print("\n" + "=" * 60)
print("UNDERWRITER AGENT - What it knows about Mike")
print("=" * 60)

underwriter_memories = underwriter.get_user_memories("mike.chen@email.com", limit=5)
for m in underwriter_memories:
    print(f"  [{m.get('category', 'general')}] {m['content']}")

## 8. Demo: Memory-Aware Follow-up

Because the Interviewer can access the Underwriter's memories (via `collaborators`), it can provide context-aware responses about past decisions.

In [ ]:
# Mike follows up later asking what he can do
response = interviewer.chat(
    user_id="mike.chen@email.com",
    message="I'm back. What can I do to improve my chances of getting approved next time?"
)

print(response)

In [ ]:
# Sarah follows up to ask about her loan
response = interviewer.chat(
    user_id="sarah.johnson@email.com",
    message="Hi again! Can you remind me what loan I was approved for and what the next steps are?"
)

print(response)

## 8b. Bidirectional Memory Sharing: Underwriter Accessing Interviewer Memories

The Underwriter also has `collaborators=["loan_interviewer"]`, meaning it can access memories from the Interviewer.

This demonstrates **bidirectional memory sharing** - both agents can see each other's stored knowledge about users.

In [ ]:
# The Underwriter directly answers a question about Sarah
# It can access the Interviewer's memories about what Sarah shared during the application
print("=" * 60)
print("UNDERWRITER ACCESSING INTERVIEWER'S MEMORIES")
print("=" * 60)

response = underwriter.chat(
    user_id="sarah.johnson@email.com",
    message="What do you know about this applicant from the interview process? What was their stated purpose for the loan and employment situation?"
)

print(response)

In [ ]:
# Similarly, the Underwriter can access Mike's application context
response = underwriter.chat(
    user_id="mike.chen@email.com",
    message="Based on what the Interviewer learned about this applicant, what were the circumstances of their loan request?"
)

print(response)

### How This Works

When the Underwriter processes a query:

1. It searches its **own** long-term memories about the user
2. It searches the **Interviewer's** long-term memories (via `collaborators`)
3. Both are injected into the context as:
   ```
   [Shared knowledge from other agents]
   Information from loan_interviewer:
   - User works as a software engineer
   - User wants loan for home improvements
   ```
4. The LLM uses this combined context to generate a response

This enables specialized agents to collaborate while maintaining their own memory stores.

## Summary

This notebook demonstrated:

1. **Multi-Agent Architecture**: Two specialized agents (Interviewer and Underwriter) with distinct personas

2. **Tool-Based Handoff**: The Interviewer delegates underwriting decisions to the Underwriter agent via the `submit_to_underwriter` tool

3. **Memory Sharing via `collaborators`**: Both agents can read each other's stored memories about users, enabling:
   - Interviewer to see past underwriting decisions
   - Underwriter to see application context gathered by Interviewer

4. **User-Friendly Communication**: Technical decisions (DTI ratios, credit scores) are translated into empathetic, plain-language responses

5. **Separate Databases**: Each agent maintains its own MongoDB database for memory isolation

### Key Patterns

```python
# Pattern 1: Memory sharing via collaborators
interviewer = MongoAgent(
    name="loan_interviewer",
    collaborators=["loan_underwriter"]  # Can read underwriter's memories
)

# Pattern 2: Tool-based agent-to-agent handoff
def submit_to_underwriter(...):
    # Call another agent's chat method
    response = underwriter.chat(user_id, message)
    return response

interviewer.register_tool(name="submit_to_underwriter", func=submit_to_underwriter, ...)
```